### LAB 5 [CLEAN COPY]

In [1]:
import sklearn
import matplotlib.pyplot as plt
from typing import Tuple
from typing import List

%matplotlib inline

import numpy as np
import pandas as pd
import scipy
import pandas as pd 
from sklearn import linear_model
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc, classification_report, precision_recall_curve, average_precision_score
import matplotlib
import matplotlib.pyplot as plt
from IPython.display import display, HTML
from sklearn.metrics import precision_recall_curve
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
run()

def run():
    years = ['2014', '2015', '2016', '2017']
    for year in years:
        

In [132]:
# df_2017 = pd.read_csv("data/2017.csv.gz",low_memory=False, encoding = "ISO-8859-1", nrows=100000, header=None)

df_2017 = pd.read_csv("data/2017_TMIN.csv", low_memory=False, encoding = "ISO-8859-1", header=None)
df_2016 = pd.read_csv("data/2016_TMIN.csv", low_memory=False, encoding = "ISO-8859-1", header=None)
df_2015 = pd.read_csv("data/2015_TMIN.csv", low_memory=False, encoding = "ISO-8859-1", header=None)
df_2014 = pd.read_csv("data/2014_TMIN.csv", low_memory=False, encoding = "ISO-8859-1", header=None)


df_2017.head()
df_2017.reset_index()
df_2014.head(10)

,0,1,2,3,4,5,6,7
0,ASN00015643,20140101,TMIN,227,NaN,NaN,a,NaN
1,ASN00085296,20140101,TMIN,131,NaN,NaN,a,NaN
2,ASN00085280,20140101,TMIN,84,NaN,NaN,a,NaN
3,CA005030984,20140101,TMIN,-315,NaN,NaN,C,NaN
4,CA003076680,20140101,TMIN,-207,NaN,NaN,C,NaN
5,CA003072151,20140101,TMIN,-258,NaN,NaN,C,NaN
6,CA003031094,20140101,TMIN,-96,NaN,NaN,C,NaN
7,ASN00068151,20140101,TMIN,172,NaN,NaN,a,NaN
8,ARM00087582,20140101,TMIN,228,NaN,NaN,S,NaN
9,USW00024061,20140101,TMIN,-93,NaN,NaN,W,NaN


In [193]:
df_2018_test = pd.read_csv("data/2018_test.csv", low_memory=False, encoding = "ISO-8859-1")
df_2018_test.head()
df_2018_test['date_mm_dd'] = create_date_mm_dd(df_2018_test['DATE'].astype('str'))
df_2018_test.head()

,ID,DATE,date_mm_dd
0,ASN00015643,20180101,0101
1,ASN00085296,20180101,0101
2,ASN00085280,20180101,0101
3,CA005030984,20180101,0101
4,CA003076680,20180101,0101


In [133]:
def rename_columns(df):
    df.rename(index=int,columns = {0:'ID',1:'date', 2:'element',3:'data_value',4:'mFlag',5:'qFlag',6:'sFlag',7:'obsTime'}, inplace=True)

In [134]:
rename_columns(df_2017)
rename_columns(df_2016)
rename_columns(df_2015)
rename_columns(df_2014)


In [135]:
def merge(df):
    return df[['ID','date']].apply(lambda x : '{}{}'.format(x[0],x[1]), axis=1)
df_2017['merged'] = merge(df_2017)
df_2016['merged'] = merge(df_2016)
df_2015['merged'] = merge(df_2015)
df_2014['merged'] = merge(df_2014)
print(df_2014.head())

            ID      date element  data_value mFlag qFlag sFlag  obsTime  \
0  ASN00015643  20140101    TMIN         227   NaN   NaN     a      NaN   
1  ASN00085296  20140101    TMIN         131   NaN   NaN     a      NaN   
2  ASN00085280  20140101    TMIN          84   NaN   NaN     a      NaN   
3  CA005030984  20140101    TMIN        -315   NaN   NaN     C      NaN   
4  CA003076680  20140101    TMIN        -207   NaN   NaN     C      NaN   

                merged  
0  ASN0001564320140101  
1  ASN0008529620140101  
2  ASN0008528020140101  
3  CA00503098420140101  
4  CA00307668020140101  


In [136]:
df_2014.columns

Index(['ID', 'date', 'element', 'data_value', 'mFlag', 'qFlag', 'sFlag',
       'obsTime', 'merged'],
      dtype='object')

In [139]:
print(len(df_2014))

672650


In [140]:
def pivot(df):
    return df.pivot_table( index=['merged','date', 'ID'], columns='element', values='data_value')
df_2017_pivoted = pivot(df_2017)
df_2016_pivoted = pivot(df_2016)
df_2015_pivoted = pivot(df_2015)
df_2014_pivoted = pivot(df_2014)

df_2017_pivoted.reset_index(inplace=True)
df_2016_pivoted.reset_index(inplace=True)
df_2015_pivoted.reset_index(inplace=True)
df_2014_pivoted.reset_index(inplace=True)

# df = df.pivot_table(
#     values='value',
#     index=['stream_name', 'preferred_timestamp', 'internal_timestamp'],
#     columns='value_id'
#     )

In [141]:
df_2017_pivoted.head()

element,merged,date,ID,TMIN
0,AE00004119620170103,20170103,AE000041196,156
1,AE00004119620170105,20170105,AE000041196,176
2,AE00004119620170107,20170107,AE000041196,130
3,AE00004119620170108,20170108,AE000041196,128
4,AE00004119620170109,20170109,AE000041196,126


In [142]:
print(df_2017_pivoted.head())
print(df_2016_pivoted.head())
print(df_2015_pivoted.head())
print(df_2014_pivoted.head())


element               merged      date           ID  TMIN
0        AE00004119620170103  20170103  AE000041196   156
1        AE00004119620170105  20170105  AE000041196   176
2        AE00004119620170107  20170107  AE000041196   130
3        AE00004119620170108  20170108  AE000041196   128
4        AE00004119620170109  20170109  AE000041196   126
element               merged      date           ID  TMIN
0        AE00004119620160101  20160101  AE000041196   155
1        AE00004119620160102  20160102  AE000041196   188
2        AE00004119620160105  20160105  AE000041196   139
3        AE00004119620160108  20160108  AE000041196   147
4        AE00004119620160109  20160109  AE000041196   136
element               merged      date           ID  TMIN
0        AE00004119620150101  20150101  AE000041196   125
1        AE00004119620150102  20150102  AE000041196   127
2        AE00004119620150103  20150103  AE000041196   140
3        AE00004119620150107  20150107  AE000041196   136
4        AE000

In [143]:
import re
def split_it(date):
    return re.findall('(\d\d\d\d)$', date)[0]

def create_date_mm_dd(date):
    return date.apply(split_it)

In [144]:
from functools import reduce
def df_merge_tmin_years(df1, df2):
   # return pd.merge(df1, df2, df3, df4, on=['ID', 'date_mm_dd'], how='outer')
    return pd.merge(df1, df2, how='outer', on=['ID', 'date_mm_dd'])
   # return reduce(lambda left,right: pd.merge(left,right,on=['ID', 'date_mm_dd'], how='outer'), dfs)

In [145]:
df_2017_pivoted['date_mm_dd'] = create_date_mm_dd(df_2017_pivoted['date'].astype('str'))
df_2016_pivoted['date_mm_dd'] = create_date_mm_dd(df_2016_pivoted['date'].astype('str'))
df_2015_pivoted['date_mm_dd'] = create_date_mm_dd(df_2015_pivoted['date'].astype('str'))
df_2014_pivoted['date_mm_dd'] = create_date_mm_dd(df_2014_pivoted['date'].astype('str'))


In [147]:
len(df_2017_pivoted)

634291

In [148]:
def tmin_rename(df, year):
    df.rename(index=str ,columns = {'TMIN':'TMIN_%s'% year}, inplace=True)

tmin_rename(df_2014_pivoted, '2014')
tmin_rename(df_2015_pivoted, '2015')
tmin_rename(df_2016_pivoted, '2016')
tmin_rename(df_2017_pivoted, '2017')

In [149]:
df_2014_pivoted.columns

Index(['merged', 'date', 'ID', 'TMIN_2014', 'date_mm_dd'], dtype='object', name='element')

In [150]:
df_compiled_2017_2016 = df_merge_tmin_years(df_2017_pivoted, df_2016_pivoted)
df_compiled_2017_2016_2015 = df_merge_tmin_years(df_compiled_2017_2016, df_2015_pivoted)
df_compiled_2017_2016_2015_2014 = df_merge_tmin_years(df_compiled_2017_2016_2015, df_2014_pivoted)


In [151]:
df_clean = df_compiled_2017_2016_2015_2014[['date_mm_dd', 'ID', 'TMIN_2014', 'TMIN_2015', 'TMIN_2016', 'TMIN_2017' ]]

In [155]:
print(df_clean.head())
print(df_clean.isnull().sum())
print(len(df_clean))


element date_mm_dd           ID  TMIN_2014  TMIN_2015  TMIN_2016  TMIN_2017
0             0103  AE000041196      140.0      140.0        NaN      156.0
1             0105  AE000041196        NaN        NaN      139.0      176.0
2             0107  AE000041196        NaN      136.0        NaN      130.0
3             0108  AE000041196        NaN        NaN      147.0      128.0
4             0109  AE000041196      115.0        NaN      136.0      126.0
element
date_mm_dd         0
ID                 0
TMIN_2014      97077
TMIN_2015     104439
TMIN_2016     117329
TMIN_2017     135436
dtype: int64
769727


In [197]:
def get_avg_2018(df):
    return df.mean(axis = 1, skipna=True)
    #return df

df_clean['mean'] = get_avg_2018(df_clean[['TMIN_2014', 'TMIN_2015', 'TMIN_2016', 'TMIN_2017' ]])



/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [198]:
print(len(df_2018))
print(len(df_clean))

769727
769727


In [201]:
df_clean.isnull().sum()

element
date_mm_dd         0
ID                 0
TMIN_2014      97077
TMIN_2015     104439
TMIN_2016     117329
TMIN_2017     135436
mean               0
dtype: int64

In [208]:
# Combine 2018 test data with average
df_pred = pd.merge(df_2018_test, df_clean, how='left', on=['ID', 'date_mm_dd'])
df_pred[df_pred['mean'].isnull()]


397804

In [222]:
df_pred_formatted = df_pred[['ID', 'DATE', 'mean']]
df_pred_formatted['mean'].fillna(0, inplace=True)
df_pred_formatted['SUB_ID'] = df_pred_formatted[['DATE','ID']].apply(lambda x : '{}{}'.format(x[0],x[1]), axis=1)


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/generic.py:3660: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [228]:
df_final = df_pred_formatted[['SUB_ID', 'mean']]

In [231]:
df_final.tail()

,SUB_ID,mean
397799,20180220USC00304174,-100.0
397800,20180220USC00246647,-44.5
397801,20180220USC00094170,25.0
397802,20180220ROE00108889,12.0
397803,20180220USC00086092,116.5


In [232]:
df_final.to_csv('dma_submission_1.csv')